# Explore here

In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, r2_score

In [3]:
origin = "https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv"


# 2 Nombre y ruta de la carpeta para datos en crudo
raw_data_folder = "data/raw"
raw_data_path = os.path.join(raw_data_folder, "demographic_health_data.csv")
 
# Crear la carpeta si no existe
if not os.path.exists(raw_data_folder):
    os.makedirs(raw_data_folder)

# Petición para descargar el fichero
response = requests.get(origin)

# Verificar si la descarga fue exitosa
if response.status_code == 200:
    # Guardar el contenido en la carpeta en crudo
    with open(raw_data_path, 'wb') as file:
        file.write(response.content)
    print("Archivo guardado en crudo exitosamente en:", raw_data_path)
else:
    print("Error al descargar el archivo:", response.status_code)

# Leer el archivo CSV descargado 
df = pd.read_csv(raw_data_path)

Archivo guardado en crudo exitosamente en: data/raw/demographic_health_data.csv


In [6]:
df.shape

(3140, 108)

In [5]:
df.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [31]:
df.info("columns")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 108 columns):
 #    Column                                                                         Dtype  
---   ------                                                                         -----  
 0    fips                                                                           int64  
 1    TOT_POP                                                                        int64  
 2    0-9                                                                            int64  
 3    0-9 y/o % of total pop                                                         float64
 4    19-Oct                                                                         int64  
 5    10-19 y/o % of total pop                                                       float64
 6    20-29                                                                          int64  
 7    20-29 y/o % of total pop                         

In [17]:
# Filtrar columnas de tipo object
object_columns = df.select_dtypes(include=['object'])

# Mostrar información de esas columnas
object_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   COUNTY_NAME  3140 non-null   object
 1   STATE_NAME   3140 non-null   object
dtypes: object(2)
memory usage: 49.2+ KB


In [22]:
# Filtrar columnas de tipo object
float_columns = df.select_dtypes(include=['float64'])

# Mostrar información de esas columnas
float_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 61 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   0-9 y/o % of total pop                                                         3140 non-null   float64
 1   10-19 y/o % of total pop                                                       3140 non-null   float64
 2   20-29 y/o % of total pop                                                       3140 non-null   float64
 3   30-39 y/o % of total pop                                                       3140 non-null   float64
 4   40-49 y/o % of total pop                                                       3140 non-null   float64
 5   50-59 y/o % of total pop                                                       3140 non-null   float64
 6   60-69 y/o % of total pop

In [18]:
df["COUNTY_NAME_n"] = pd.factorize(df["COUNTY_NAME"])[0]
df["STATE_NAME_n"] = pd.factorize(df["STATE_NAME"])[0]

In [23]:
non_numeric_columns = df.select_dtypes(include=['object', 'category']).columns
print("Columnas no numéricas:", non_numeric_columns)

Columnas no numéricas: Index(['COUNTY_NAME', 'STATE_NAME'], dtype='object')


In [7]:
total_data = df.drop_duplicates().reset_index(drop = True)
total_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [19]:
# Dividimos el conjunto de datos en muestras de train y test
X = df.drop("diabetes_number", axis = 1)
y = df["diabetes_number"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,COUNTY_NAME_n,STATE_NAME_n
1292,26127,26625,3221,12.097653,3463,13.006573,2922,10.974648,2829,10.625352,...,13.7,12.6,14.9,3.8,3.5,4.1,771,6,879,22
2302,42121,51266,5272,10.283619,5751,11.217961,5137,10.020286,5341,10.418211,...,13.1,11.9,14.2,3.5,3.2,3.8,1454,5,1404,38
761,18133,37779,3915,10.362900,5118,13.547209,6202,16.416528,4363,11.548744,...,12.2,11.2,13.1,2.9,2.7,3.1,871,2,327,14
2194,40131,91984,11163,12.135806,12646,13.748043,11595,12.605453,11357,12.346712,...,11.2,10.4,12.0,3.0,2.8,3.2,2118,3,1350,36
1241,26025,134487,16698,12.416070,17666,13.135842,17281,12.849569,15993,11.891856,...,12.5,11.7,13.4,3.4,3.2,3.6,3490,4,7,22


In [20]:
# seleccion de variables
X_train = df.drop(["diabetes_number"], axis = 1)
y_train = df["diabetes_number"]

X_test = df.drop(["diabetes_number"], axis = 1)
y_test = df["diabetes_number"]

In [26]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for col in non_numeric_columns:
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])


In [27]:
# modelo
reg_lin = LinearRegression()

# entrenamiento
reg_lin.fit(X_train, y_train)

# predicción
y_pred = reg_lin.predict(X_test)
y_pred

# metricas
mse_rl = mean_squared_error(y_test, y_pred)
rmse_rl = np.sqrt(mse_rl)
r2_rl = r2_score(y_test, y_pred)

print("MSE: ", mse_rl)
print("RMSE: ", rmse_rl)
print("Coeficiente de determinación: ", r2_rl)

MSE:  987934.1862206105
RMSE:  993.9487845058268
Coeficiente de determinación:  0.9988837587203455


El resultado del RMSE es demasiado alto, voy a intentar aplicar el mismo modelo pero normalizando los datos

In [30]:
media = df["diabetes_number"].mean()
moda = df["diabetes_number"].mode()
Var = df["diabetes_number"].var()

print(media, moda, Var)

9326.577707006369 0    547
Name: diabetes_number, dtype: int64 885336291.7024658


In [31]:
#modelo
reg_lasso = Lasso(alpha = 0.001, max_iter = 8000)

# entrenamiento
reg_lasso.fit(X_train, y_train)

# prediccion
y_pred = reg_lasso.predict(X_test)
y_pred

# metricas
mse_l1 = mean_squared_error(y_test, y_pred)
rmse_l1 = np.sqrt(mse_l1)
r2_l1 = r2_score(y_test, y_pred)

print("MSE: ", mse_l1)
print("RMSE: ", rmse_l1)
print("Coeficiente de determinación: ", r2_l1)

MSE:  1071375.0719963457
RMSE:  1035.0724960099876
Coeficiente de determinación:  0.998789481022081


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.682e+09, tolerance: 2.779e+08
  model = cd_fast.enet_coordinate_descent(


In [32]:
# modelo
reg_ridge = Ridge(alpha = 0.0000000001, max_iter = 400)

# entrenamiento
reg_ridge.fit(X_train, y_train)

# prediccion
y_pred = reg_ridge.predict(X_test)
y_pred

# metricas
mse_l2 = mean_squared_error(y_test, y_pred)
rmse_l2 = np.sqrt(mse_l2)
r2_l2 = r2_score(y_test, y_pred)

print("MSE: ", mse_l2)
print("RMSE: ", rmse_l2)
print("Coeficiente de determinación: ", r2_l2)

MSE:  981721.7904292665
RMSE:  990.8187475160462
Coeficiente de determinación:  0.9988907779456386


In [33]:
# Datos
data = {
    'Modelo': ['Regresión lineal', "Regulación Lasso", "Regulación Ridge"],
    'MSE': [mse_rl, mse_l1, mse_l2],
    'RMSE': [rmse_rl, rmse_l1, rmse_l2],
    'R²': [r2_rl, r2_l1, r2_l2]
}

# Crear el DataFrame
resultados = pd.DataFrame(data)

# Mostrar el DataFrame
print(resultados)

             Modelo           MSE         RMSE        R²
0  Regresión lineal  9.879342e+05   993.948785  0.998884
1  Regulación Lasso  1.071375e+06  1035.072496  0.998789
2  Regulación Ridge  9.817218e+05   990.818748  0.998891
